This notebook will be used to redesign the tidal ellipse functions to make them more user friendly

In [1]:
import os
import datetime
import matplotlib.pylab as plt
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import seaborn as sns
from dateutil import tz
from scipy.optimize import curve_fit
import collections

import netCDF4 as nc
from salishsea_tools import (viz_tools, tidetools as tt, ellipse, nc_tools)
from salishsea_tools.nowcast import (analyze, research_VENUS)


%matplotlib inline

#Some of the functions
The functions below will be in the ellipse.py module. They are meant to be versitile to different file types and to be user friendly. In order to be user friendly they must be able to cover many diffrent types of ellipse calculations and be well commented to clearly show what is going on in the scripts. Also there are many scripts with small tasks to faciliate adding in new scripts for a modification.

In [2]:
def quadruple(x, M2amp, M2pha, K1amp, K1pha, S2amp, S2pha, O1amp, O1pha, mean):
    """Function for the fit, assuming only 4 constituents of importance are: M2, K2, S1 and O1.

    :arg x: Independant variable, time.
    :type x:
    
    :arg *amp: Tidal amplitude of the a constituent
    :type *amp: float

    :arg *pha: Phase lag of a constituent
    :type *pha: float
    
    :returns: function for fitting 4 frequencies
    """
    return(
        mean + M2amp * np.cos(ellipse.CorrTides['M2']['freq'] * x - M2pha * np.pi / 180) +
        K1amp * np.cos(ellipse.CorrTides['K1']['freq'] * x - K1pha * np.pi / 180) + 
        S2amp * np.cos(ellipse.CorrTides['S2']['freq'] * x - S2pha * np.pi / 180) +
        O1amp * np.cos(ellipse.CorrTides['O1']['freq'] * x - O1pha * np.pi / 180))

In [3]:
def sextuple(x, M2amp, M2pha, K1amp, K1pha, S2amp, S2pha, O1amp, O1pha, N2amp, N2pha, P1amp, P1pha, mean):
    """Function for the fit, assuming 6 constituents of importance are: M2, K2, S1, O1, N2 and P1.


    :arg x: Independant variable, time.
    :type x:
    
    :arg *amp: Tidal amplitude of the a constituent
    :type *amp: float

    :arg *pha: Phase lag of a constituent
    :type *pha: float

    :returns:function for fitting 6 frequencies
    """
    return(
        mean + M2amp * np.cos(ellipse.CorrTides['M2']['freq'] * x - M2pha * np.pi / 180) +
        K1amp * np.cos(ellipse.CorrTides['K1']['freq'] * x - K1pha * np.pi / 180) + 
        S2amp * np.cos(ellipse.CorrTides['S2']['freq'] * x - S2pha * np.pi / 180) +
        O1amp * np.cos(ellipse.CorrTides['O1']['freq'] * x - O1pha * np.pi / 180) +
        N2amp * np.cos(ellipse.CorrTides['N2']['freq'] * x - N2pha * np.pi / 180) +
        P1amp * np.cos(ellipse.CorrTides['P1']['freq'] * x - P1pha * np.pi / 180))

In [4]:
def octuple(x, M2amp, M2pha, K1amp, K1pha, S2amp, S2pha, O1amp, O1pha, N2amp, N2pha, P1amp, P1pha, K2amp, K2pha, Q1amp, Q1pha,mean):
    """Function for the fit, for all the constituents: M2, K2, S1, O1, N2, P1, K2, Q1.

    :arg x: Independant variable, time.
    :type x:
    
    :arg *amp: Tidal amplitude of the a constituent
    :type *amp: float

    :arg *pha: Phase lag of a constituent
    :type *pha: float

    :returns: function for fitting 8 frequencies
    """
    return(
        mean + M2amp * np.cos(ellipse.CorrTides['M2']['freq'] * x - M2pha * np.pi / 180) +
        K1amp * np.cos(ellipse.CorrTides['K1']['freq'] * x - K1pha * np.pi / 180) + 
        S2amp * np.cos(ellipse.CorrTides['S2']['freq'] * x - S2pha * np.pi / 180) +
        O1amp * np.cos(ellipse.CorrTides['O1']['freq'] * x - O1pha * np.pi / 180) +
        N2amp * np.cos(ellipse.CorrTides['N2']['freq'] * x - N2pha * np.pi / 180) +
        P1amp * np.cos(ellipse.CorrTides['P1']['freq'] * x - P1pha * np.pi / 180) +
        K2amp * np.cos(ellipse.CorrTides['K2']['freq'] * x - K2pha * np.pi / 180) +
        Q1amp * np.cos(ellipse.CorrTides['Q1']['freq'] * x - Q1pha * np.pi / 180))

In [5]:
def get_params(u, v, time, nconst, tidecorr=ellipse.CorrTides):
    """Calculates tidal ellipse parameters from the u and v time series. Maintains the shape 
    of the velocities enters only loosing the time dimensions.
    
    :arg u: One of the orthogonal tidal current velocities. Must be already prepared for the analysis.
    :type u:  :py:class:'np.ndarray'
    
    :arg v: One of the orthogonal tidal current velocities. Must be already prepared for the analysis.
    :type v:  :py:class:'np.ndarray'
    
    :arg time: Time over which the velocities were taken; in seconds.
    :type time: :py:class:'np.ndarray'
    
    :arg tidecorr: Tidal corrections in aplitude and phase. Default is the nowcast values. 
    :type tidecorr: dictionary
    """
    params = {}
    
    #Running fittit to get the amplitude and phase of the velcity time series.
    uapparam = fittit(u, time, nconst)
    vapparam = fittit(v, time, nconst)
    #Cycling through the constituents in the ap-parameter dict given by fittit
    for const in uapparam:

        #Applying tidal corrections to u and v phase parameter
        uapparam[const]['phase'] = uapparam[const]['phase'] + tidecorr[const]['uvt']
        vapparam[const]['phase'] = vapparam[const]['phase'] + tidecorr[const]['uvt']
        #Applying tidal corrections to u and v aplitude parameter
        uapparam[const]['amp'] = uapparam[const]['amp'] * tidecorr[const]['ft']
        vapparam[const]['amp'] = vapparam[const]['amp'] * tidecorr[const]['ft']
        
        #Converting from u/v amplitude and phase to ellipe parameters. Inputs are the amplitude and phase of both velocities, runs once for each contituent
        CX, SX, CY, SY, ap, am, ep, em, maj, mi, the, pha = tt.ellipse_params(uapparam[const]['amp'], uapparam[const]['phase'], vapparam[const]['amp'], vapparam[const]['phase'])
        
        #Filling the dictionary with ep-parameters given by ellipse_param. Each constituent will be a different key.
        params[const] = {'Semi-Major Axis' : maj,
                        'Semi-Minor Axis': mi,
                        'Inclination': the,
                        'Phase': pha}
    
    return params
    
    

In [6]:
def fittit(uaus, time, nconst):
    """Function to find tidal components from a tidal current component
        over the whole area given. Can be done over depth, or an area.
        Time must be in axis one, depth in axis two if applicable then the
        x and y if an area.
        Must perform twice, once for each tidal current vector
        in order to complete the analysis.
        In order to calculate the tidal components of an area at a single
        depth the velocity vector must only have 3 dimensions. For a depth
        profile it must only have 2 dimensions 
        ***[time, depth, x, y]

    :arg uaus: One of the orthogonal tidal current velocities.
    :type uaus:  :py:class:'np.ndarray' or float

    :arg time: Time over which the velocitie were being taken in seconds.
    :type time: :py:class:'np.ndarray'

    :returns M2amp, M2pha, K1amp, K1pha:
        The amplitude and phase lag of each tidal component (M2 and K1)
        of a single tidal velocity vector.

    """
    apparam = collections.OrderedDict()
    apparam['M2'] = {'amp': [], 'phase': []}
    apparam['K1'] = {'amp': [], 'phase': []}
    fitfunction = tt.double
    if nconst >2:
        fitfunction = quadruple
        apparam['S2'] = {'amp': [], 'phase': []}
        apparam['O1'] = {'amp': [], 'phase': []}    
    if nconst >4:
        fitfunction = sextuple
        apparam['N2'] = {'amp': [], 'phase': []}
        apparam['P1'] = {'amp': [], 'phase': []}    
    if nconst >6:
        fitfunction = octuple
        apparam['K2'] = {'amp': [], 'phase': []}
        apparam['Q1'] = {'amp': [], 'phase': []}  
    
    #Case 1: a time series of velocities with depth at a single location.
    if uaus.ndim == 2:
        #The parameters are the same shape as the velocities without the time dimension.
        thesize = uaus.shape[1]
        for const, ap in apparam.iteritems():
             for key2 in ap:
                    ap[key2] = np.zeros(thesize)
        
        #Calculates the parameters for one depth and one location at a time from its time series
        for dep in np.arange(0, len(uaus[1])):
            if uaus[:, dep].any() != 0:
                fitted, cov = curve_fit(fitfunction, time[:], uaus[:, dep])
                #Rotating to have a positive amplitude and a phase between [-180, 180]
                for k in np.arange(nconst):
                    fitted[2*k], fitted[2*k+1] = tt.convention_pha_amp(fitted[2*k], fitted[2*k+1])
              
                for const, k in zip(apparam, np.arange(0,nconst)):
                    apparam[const]['amp'][dep] = fitted[2*k]
                    apparam[const]['phase'][dep] = fitted[2*k+1]

    #Case 2 : a time series of an area of velocities at a single depth
    elif uaus.ndim == 3:
        thesize = (uaus.shape[1], uaus.shape[2])
        for const, ap in apparam.iteritems():
             for key2 in ap:
                    ap[key2] = np.zeros(thesize)
                    
        for i in np.arange(0, uaus.shape[1]):
            for j in np.arange(0, uaus.shape[2]):
                if uaus[:, i, j].any() != 0.:
                    fitted, cov = curve_fit(fitfunction, time[:], uaus[:, i, j])
                    for k in np.arange(nconst):
                        fitted[2*k], fitted[2*k+1] = tt.convention_pha_amp(fitted[2*k], fitted[2*k+1])
                    
                for const, k in zip(apparam, np.arange(0,nconst)):
                    apparam[const]['amp'][i, j] = fitted[2*k]
                    apparam[const]['phase'][i, j] = fitted[1*k+1]
    
    #Case 3: a time series of an area of velocities with depth
    elif uaus.ndim == 4:
        thesize = (uaus.shape[1], uaus.shape[2], uaus.shape[3])
        for const, ap in apparam.iteritems():
             for key2 in ap:
                    ap[key2] = np.zeros(thesize)

        for dep in np.arange(0, uaus.shape[1]):
            for i in np.arange(0, uaus.shape[2]):
                for j in np.arange(0, uaus.shape[3]):
                    if uaus[:, dep, i, j].any() != 0.:
                        fitted, cov = curve_fit(fitfunction, time[:], uaus[:, dep, i, j])
                        for k in np.arange(nconst):
                            fitted[2*k], fitted[2*k+1] = tt.convention_pha_amp(
                                fitted[2*k], fitted[2*k+1])
                            
                        for const, k in zip(apparam, np.arange(0,nconst)):
                            apparam[const]['amp'][dep, i, j] = fitted[2*k]
                            apparam[const]['phase'][dep, i, j] = fitted[2*k+1]

    #Case 4: a time series of a single location with a single depth.
    else:
        thesize = (0)
        for const, ap in apparam.iteritems():
             for key2 in ap:
                    ap[key2] = np.zeros(thesize)

        if uaus[:].any() != 0.:
            fitted, cov = curve_fit(fitfunction, time[:], uaus[:])
            for k in np.arange(nconst):
                fitted[2*k], fitted[2*k+1] = tt.convention_pha_amp(
                    fitted[2*k], fitted[2*k+1])
            for const, k in zip(apparam, np.arange(0,nconst)):
                apparam[const]['amp'] = fitted[2*k]
                apparam[const]['phase'] = fitted[2*k+1]
    return apparam

##Case 1: Nowcast date range
Functions for nowcast velocities.

In [7]:
def ellipse_files_nowcast(to, tf, iss, jss, path, depthrange='None',reftime=ellipse.CorrTides['reftime']):
    """ This function loads all the data between the start and the end date
    that contains hourly velocities in the netCDF4 nowcast files in the specified depth range. 
    This will make an area with all the indices indicated, the area must be continuous for unstaggering.

    :arg to: The beginning of the date range of interest
    :type to: datetime object

    :arg tf: The end of the date range of interest
    :type tf: datetime object
    
    :arg iss: x index.
    :type i: list or numpy.array
    
    :arg jss: y index.
    :type j: list or numpy.array

    :arg path: Defines the path used(eg. nowcast)
    :type path: string

    :arg depthrange: Depth values of interest in meters as a float for a single depth or a list for a range. 
        A float will find the closest depth that is <= the value given. Default is 'None' for the whole
        water column (0-441m).
    :type depav: float, string or list.

    :returns: u, v, time, dep.
    """
    
    #The unstaggering in prepare_vel.py requiers an extra i and j, we add one on here to maintain the 
    #area, or point chosen.
    assert np.logical_and(type(iss)!=int, type(jss)!=int), 'Must enter list, even if only looking at a single value'
    assert np.logical_and(iss[0] != 0, jss[0] != 0), 'Cannot begin analysis on x=0 or y=0.'
    
    jss = np.append(jss[0]-1,jss)
    iss = np.append(iss[0]-1,iss)
    
    #Makes a list of the filenames that follow the criteria in the indicated path between the start and end dates.
    filesu = analyze.get_filenames(to, tf, '1h', 'grid_U', path) 
    filesv = analyze.get_filenames(to, tf, '1h', 'grid_V', path)
    
    #Set up depth array and depth range
    depth = nc.Dataset(filesu[-1]).variables['depthu'][:]
    
    #Case one: for a single depth.
    if type(depthrange) == float or type(depthrange) == int:
        k = np.where(depth <= depthrange)[0][-1]
        u, time = analyze.combine_files(filesu, 'vozocrtx', k, jss, iss)
        v, time = analyze.combine_files(filesv, 'vomecrty', k,  jss, iss)
        dep=depth[k]
    
    #Case two: for a specific range of depths
    elif type(depthrange) == list:
        k = np.where(np.logical_and(depth > depthrange[0], depth < depthrange[1]))[0]
        dep = depth[k]
        u, time = analyze.combine_files(filesu, 'vozocrtx', k, jss, iss)
        v, time = analyze.combine_files(filesv, 'vomecrty', k,  jss, iss)
    
    #Case three: For the whole depth range 0 to 441m.
    else:
        u, time = analyze.combine_files(filesu, 'vozocrtx', depthrange, jss, iss)
        v, time = analyze.combine_files(filesv, 'vomecrty', depthrange,  jss, iss)
        dep=depth
        
    #For the nowcast the reftime is always Sep10th 2014. Set time of area we are looking at relative to this time.
    time = tt.convert_to_seconds(time, reftime=reftime)
    
    return u, v, time, dep

In [8]:
def prepare_vel(u, v, depav=False, depth='None'):
    """Preparing the time series of the orthogonal pair of velocities to get tidal ellipse parameters. 
    This function masks, rotates and unstaggers the time series. The depth averaging does not work over 
    masked values.
    
    :arg u: One of the orthogonal components of the time series of tidal current velocities.
    :type u:  :py:class:'np.ndarray'
    
    :arg v: One of the orthogonal components of the time series of tidal current velocities. 
    :type v:  :py:class:'np.ndarray'

    :arg depav: True will depth average over the whole depth profile given. Default is False.
    :type dep: boolean
    
    :arg depth: depth vector corresponding to the depth of the velocities, only requiered if depav=True.
    :type depth: :py:class:'np.ndarray' or string
    """
    #Masks land values
    u_0 = np.ma.masked_values(u, 0)
    v_0 = np.ma.masked_values(v, 0)

    #Unstaggers velocities. Will loose one x and one y dimension due to unstaggering.
    u_u, v_v = research_VENUS.unstag_rot(u_0, v_0)

    #Depth averaging over all the depth values given if set to True.
    if depav == True:
        assert type(depth)!=str, 'Depth values must be entered to proceed with depth averaging'
        u_u = analyze.depth_average(u_u, depth, 1)
        v_v = analyze.depth_average(v_v, depth, 1)
    return u_u, v_v

In [9]:
def get_params_nowcast(to, tf, i, j, path, nconst, depthrange='None', depav=False, tidecorr=ellipse.CorrTides):
    """ This function loads all the data between the start and the end date
    that contains hourly velocities in the netCDF4 nowcast files in the specified depth range. Then
    masks, rotates and unstaggers the time series. The unstaggering causes the shapes of 
    the returned arrays to be 1 less than those of the input arrays in the y and x dimensions. Finally it
    calculates tidal ellipse parameters from the u and v time series. Maintains the shape 
    of the velocities enters only loosing the time dimensions.

    :arg to: The beginning of the date range of interest
    :type to: datetime object

    :arg tf: The end of the date range of interest
    :type tf: datetime object

    :arg i: x index, must have at least 2 values for unstaggering, will loose the first i 
        during the unstaggering in prepare_vel.
    :type i: float or list
    
    :arg j: y index, must have at least 2 values for unstaggering, will loose the first j 
        during the unstaggering in prepare_vel.
    :type j: float or list

    :arg path: Defines the path used(eg. nowcast)
    :type path: string

    :arg depthrange: Depth values of interest in meters as a float for a single depth or a list for a range. 
            A float will find the closest depth that is <= the value given. Default is 'None' for the whole
            water column (0-441m).
    :type depav: float, string or list.

    :arg depav: True will depth average over the whole depth profile given. Default is False.
    :type dep: boolean
    
    :arg depth: depth vector corresponding to the depth of the velocities, only requiered if depav=True.
    :type depth: :py:class:'np.ndarray' or string
    
    :returns: params, a numpy array containting [M2 semi-major, M2 semi-minor, M2 inclication, M2 phase, K1 semi-major, 
            K1 semi-minor, K1 inclication, K1 phase]
    """
        
    u, v, time, dep = ellipse_files_nowcast(to, tf, i, j, path, depthrange=depthrange)
    u_u, v_v = prepare_vel(u, v, depav=depav, depth=dep)
    params = get_params(u_u, v_v, time, nconst, tidecorr=tidecorr)
    
    return params, dep

###Nowcast runs: single location, single depth

In [19]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'


#The longer the date range the longer the analysis takes
to=datetime.datetime(2015,7,10)
tf=datetime.datetime(2015,7,11)

i = [280]
j = [400]


In [20]:
u, v, time, dep = ellipse_files_nowcast(to, tf, i, j, path, depthrange=5)
u_u, v_v = prepare_vel(u, v, depav=False, depth=dep)
params = get_params(u_u, v_v, time, 4)
print params['S2']['Semi-Major Axis']

[[ 0.05098803]]


In [21]:
u, v, time, dep = ellipse_files_nowcast(to, tf, i, j, path, depthrange=5)
u_u, v_v = prepare_vel(u, v, depav=False, depth=dep)
params = get_params(u_u, v_v, time, 6)
params['S2']['Semi-Major Axis']

array([[ 0.04117673]])

In [22]:
u, v, time, dep = ellipse_files_nowcast(to, tf, i, j, path, depthrange=5)
u_u, v_v = prepare_vel(u, v, depav=False, depth=dep)
params = get_params(u_u, v_v, time, 8)
params['S2']['Semi-Major Axis']

array([[ 0.32734833]])

###Nowcast runs: Area with a depth range of [0,60]

In [28]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,7,8)
tf=datetime.datetime(2015,7,11)

i = np.arange(280,283)
j = np.arange(400, 416)

In [29]:
params, dep = ellipse.get_params_nowcast(to, tf, i, j, path, 6, depthrange=[5, 60], depav=True)
print params['N2']['Phase']

[[ 189.27457484  187.70113406  206.45501458]
 [ 198.18550618  202.67500481  223.26356999]
 [ 212.87483313  215.86656965  230.48675465]
 [ 224.42610352  226.45825899  234.69223479]
 [ 227.36138011  231.73498123  237.49509332]
 [ 226.29081745  233.92830576  239.24210028]
 [ 221.53280268  232.38665557  235.95929469]
 [ 213.42171785  228.58487151  230.55522257]
 [ 204.38173983  220.96982534  225.04936683]
 [ 197.32273749  206.63588608  214.24625242]
 [ 193.42391994  197.79957108  206.63527051]
 [ 191.23950577  196.15650077  206.18917381]
 [ 189.61830514  196.44608472  206.55934755]
 [ 187.14486357  195.48197675  204.99338425]
 [ 186.4770142   195.33422055  203.48690103]
 [ 186.9653332   195.91976477  202.66835621]]


#Case 2: Any other type of files

For any other type of file, there needs to be two orthorgonal time series velocitiy vectors. Each must have time in the first dimension, depth in the second if applicable and the locations in the third (and fourth if looking at an area).

The analysis will be done over the whole time, depth and area that is given.

U and V must be the same shape.

Time vector must be in seconds...

In [34]:
grid = '/ocean/mdunn/MEOPAR/jun_VENUS_east_gridded.nc'
G = nc.Dataset(grid)
u = G.variables['vozocrtx'][:]
v = G.variables['vomecrty'][:]
dep = G.variables['depthu'][:]

#Set up time in seconds
t = G.variables['time_counter'][:]
time = ([])
t = nc_tools.timestamp(G, np.arange(u.shape[0]))
for ind in range(len(t)):
    t[ind] = t[ind].datetime
time = np.append(time, t)

time = tt.convert_to_seconds(time, reftime=ellipse.CorrTides['reftime'])

In [36]:
u_u,v_v = ellipse.prepare_vel(u, v, depav=True, depth=dep)
params = get_params(u_u,v_v,time, 2)
print params['K1']['Phase']

[[ 262.82680652]]


In [37]:
u_u,v_v = ellipse.prepare_vel(u, v, depav=True, depth=dep)
params = get_params(u_u,v_v,time, 8)
print params['K1']['Phase']

[[ 262.63901909]]
